In [132]:
import os
from os import walk
import pandas as pd
import tinytag
from tinytag import TinyTag
from pymongo import MongoClient
import pymongo


##  Probando walk y TinyTag 
- Walk nos servirá para hacer datasets con nuestros archivos (paths y nombre)
- TinyTag extrae la información de los metadatos de los archivos musicales


In [2]:
os.listdir('../../../Music') #encontramos la carpeta con nuestros archivos musicales

['Britpop, Indie y rock alt británico',
 'Cantautores',
 'Classic Punk, New Wave, Garage Rock y psicodelia',
 'Classic Rock',
 'Country, Folk, singersongwritters',
 'desktop.ini',
 'Electronica, House, Chill out & Inndustrial',
 'Flamenco, Fusión, Música del mundo, ritmos latinos y Francia',
 'Funk, Jazz, Blues, Swing y Rock and roll',
 'Hip hop, funky, R&B',
 'Indie en español',
 'Indie, Rock Alt, Grunge',
 'Metal, hard rock',
 'Pop Rock en español y movida',
 'Pop, Pop Dance, Club',
 'Punky, rap metal, Ska y Reggae',
 'Rock Ibérico, punkarra y rock latino',
 'V.A']

In [3]:
total = [] #lista vacía para crear la lista de diccionarios con la que crearemos posteriormente un dataframe
for (path, ficheros, archivos) in walk("../../../Music"): # iteramos sobre los archivos de la carpeta Music y guardamos su info en las variables path, ficheros y archivos (path es la ruta, ficheros son las carpetas, archivos es nombre de archivos)
    dicc = {}
    dicc['path'] =  path
    dicc['ficheros'] = ficheros
    dicc['archivos'] = archivos
    total.append(dicc)
    #print(dicc['path'])

In [4]:
total[0] #ejemplo diccionario de la lista d

{'path': '../../../Music',
 'ficheros': ['Britpop, Indie y rock alt británico',
  'Cantautores',
  'Classic Punk, New Wave, Garage Rock y psicodelia',
  'Classic Rock',
  'Country, Folk, singersongwritters',
  'Electronica, House, Chill out & Inndustrial',
  'Flamenco, Fusión, Música del mundo, ritmos latinos y Francia',
  'Funk, Jazz, Blues, Swing y Rock and roll',
  'Hip hop, funky, R&B',
  'Indie en español',
  'Indie, Rock Alt, Grunge',
  'Metal, hard rock',
  'Pop Rock en español y movida',
  'Pop, Pop Dance, Club',
  'Punky, rap metal, Ska y Reggae',
  'Rock Ibérico, punkarra y rock latino',
  'V.A'],
 'archivos': ['desktop.ini']}

In [6]:
pd.DataFrame(total).sample(5) #ejemplo de dataframe con la ruta, los ficheros y los archivos

,path,ficheros,archivos
862,"../../../Music/Country, Folk, singersongwritte...",[],"[01 Cracking.mp3, 02 Freeze Tag.mp3, 03 Marlen..."
2434,"../../../Music/Rock Ibérico, punkarra y rock l...","[1997 Radical Sonora, 1999 Pequeño, 2002 Flami...",[]
829,"../../../Music/Country, Folk, singersongwritte...",[2010 Downtown Church],[]
62,"../../../Music/Britpop, Indie y rock alt britá...",[],"[01 Jacqueline.m4a, 02 Tell Her Tonight.m4a, 0..."
1727,"../../../Music/Indie, Rock Alt, Grunge/Sonic Y...","[1986 Evol, 1987 Sister, 1988 Daydream nation,...",[]


### Necesitamos crear una línea por cada archivo

In [7]:
carpeta_m = pd.DataFrame(total) #datos de la carpeta Musica de mi pc

In [8]:
carpeta_m.sample()

,path,ficheros,archivos
1995,../../../Music/Pop Rock en español y movida/Al...,[],"[01 Los Dos Cogidos De La Mano.mp3, 02 Pisando..."


In [12]:
carpeta_m.ficheros.value_counts().head()
#la mayoría de los elementos son una lista vacía. Ya que lo que se puede llamar fichero son subcarpetas en las que están los archivos
# solo nos quedamos con el path que incluirá estos ficheros.
carpeta_m.drop('ficheros',axis=1,inplace=True)

In [13]:
carpeta_m.sample()

,path,archivos
1693,"../../../Music/Indie, Rock Alt, Grunge/Nick Ca...","[01 Into My Arms.mp3, 02 Lime Tree Arbour.mp3,..."


In [23]:
# iteramos por dataframe para crear una nueva lista de diccionarios con la ruta para cada archivo
archivos = [] #lista vacía para el diccionario archio = ruta
for i,r in carpeta_m.iterrows(): # todos los elementos de archivos son listas (las vacías las descartamos)
    if len(r.archivos)>=0:
        for arch in r.archivos:
            dicc = {}
            dicc['ruta'] = r.path
            dicc['archivo'] = arch
            archivos.append(dicc)


In [25]:
archivos_df = pd.DataFrame(archivos)

In [26]:
archivos_df.head()

,ruta,archivo
0,../../../Music,desktop.ini
1,"../../../Music/Britpop, Indie y rock alt britá...",desktop.ini
2,"../../../Music/Britpop, Indie y rock alt britá...",01 Stuck On The Puzzle (Intro).mp3
3,"../../../Music/Britpop, Indie y rock alt britá...",02 Hiding Tonight.mp3
4,"../../../Music/Britpop, Indie y rock alt britá...",03 Glass In The Park.mp3


In [85]:
# vamos a estuidar los archivos para quitar los que nos on musicales
# sacaremos la extensión del archivo para comproboarlo. (caracteres despúes del último punto)
archivos_df['extension'] = archivos_df.archivo.str.extract(r'((?<=\.)[^.]*$)')
#transformamos extensiones todoas a minusculaa
archivos_df.extension = archivos_df.extension.str.lower()
archivos_df.extension.value_counts()

mp3                       21998
jpg                        1972
db                          141
wma                          51
m4a                          48
bridgelabelsandratings       32
ini                          30
ds_store                      8
html                          3
txt                           2
log                           1
cue                           1
ico                           1
png                           1
m3u                           1
nfo                           1
bmp                           1
Name: extension, dtype: int64

In [101]:
ext_music = ['mp3','wma','m4a'] #lista de extensiones musicales para dropear los archivos que no lo son

In [102]:
archivos_mus = archivos_df.copy()[archivos_df.extension.isin(ext_music)]

In [103]:
archivos_mus.head() #dataframe con archivos musicales solo

,ruta,archivo,extension
2,"../../../Music/Britpop, Indie y rock alt britá...",01 Stuck On The Puzzle (Intro).mp3,mp3
3,"../../../Music/Britpop, Indie y rock alt britá...",02 Hiding Tonight.mp3,mp3
4,"../../../Music/Britpop, Indie y rock alt britá...",03 Glass In The Park.mp3,mp3
5,"../../../Music/Britpop, Indie y rock alt britá...",04 It's Hard To Get Around The Wind.mp3,mp3
6,"../../../Music/Britpop, Indie y rock alt britá...",05 Stuck On The Puzzle.mp3,mp3


In [104]:
archivos_mus.extension.unique()

array(['mp3', 'm4a', 'wma'], dtype=object)

## Utilizamos tinytag para extraer la info de los archivos

###  Tinytag Documentation
- [Pyp](https://pypi.org/project/tinytag/)
- [GitHub](https://github.com/devsnd/tinytag)

- self._filehandler = filehandler
- self._filename = None  # for debugging purposes
- self._default_encoding = None  # allow override for some file formats
- self.filesize = filesize
- self.album = None
- self.albumartist = None
- self.artist = None
- self.audio_offset = None
- self.bitrate = None
- self.channels = None
- self.comment = None
- self.composer = None
- self.disc = None
- self.disc_total = None
- self.duration = None
- self.extra = defaultdict(lambda: None)
- self.genre = None
- self.samplerate = None
- self.title = None
- self.track = None
- self.track_total = None
- self.year = None
- self._load_image = False
- self._image_data = None
- self._ignore_errors = ignore_errors

### posibles atributos
- tag.album         # album as string
- tag.albumartist   # album artist as string
- tag.artist        # artist name as string
- tag.audio_offset  # number of bytes before audio data begins
- tag.bitrate       # bitrate in kBits/s
- tag.comment       # file comment as string
- tag.composer      # composer as string 
- tag.disc          # disc number
- tag.disc_total    # the total number of discs
- tag.duration      # duration of the song in seconds
- tag.filesize      # file size in bytes
- tag.genre         # genre as string
- tag.samplerate    # samples per second
- tag.title         # title of the song
- tag.track         # track number as string
- tag.track_total   # total number of tracks as string
- tag.year          # year or data as string
- tag.extra         # a dict of additional data

- Aditionally you can also get cover images from ID3 tags:
``` python
tag = TinyTag.get('/some/music.mp3', image=True)
image_data = tag.get_image()
```

In [105]:
# iteramos por el dataframe archivos_mus para extraer en un dataframe la info del archivo

In [117]:
len(archivos_mus)

22097

In [126]:
listatag = []
excepcionismo = []

for i,r in archivos_mus.iterrows():
    
    try:

        dicc = {}
        #tag = TinyTag.get({str(r.ruta)}+'/'+{str(r.archivo)})
        tag = TinyTag.get(r.ruta+"/"+r.archivo)
        dicc['albumartist'] = tag.albumartist
        dicc['artist'] = tag.artist
        dicc['genre'] = tag.genre
        dicc['album'] = tag.album
        dicc['year'] = tag.year
        dicc['track'] = tag.track    
        dicc['track_total'] = tag.track_total
        dicc['title'] = tag.title    
        dicc['duration'] = tag.duration
        dicc['filesize'] = tag.filesize
        dicc['audio_offset'] = tag.audio_offset
        dicc['channels'] = tag.channels
        dicc['comment'] = tag.comment
        dicc['composer'] = tag.composer
        dicc['disc_total'] = tag.disc_total
        #dicc['extra'] = tag.extra
        dicc['samplerate'] = tag.samplerate        
        dicc['archivo'] = r.archivo
        dicc['ruta'] = r.ruta        
        dicc['ruta_total'] = f'{r.ruta}/{r.archivo}'

        listatag.append(dicc)

        if i%10000 == 0:
            print(i, 'done', r.ruta, r.archivo)
    except Exception as e:
        print(e)
        print(f'cant be done: {i}, {r.ruta+"/"+r.archivo}')
        excepcionismo.append(i)
print('done, bitches')

10000 done ../../../Music/Flamenco, Fusión, Música del mundo, ritmos latinos y Francia/La cabra mecánica/1997 Cuando me suenan las tripas 02 Pégate un tiro.mp3
20000 done ../../../Music/Pop, Pop Dance, Club/Rihanna/2007 Good Girl Gone Bad 02 Push Up on Me.mp3
done, bitches


In [127]:
# cuando ya has usado tinytag se queda grabado los datos en meororia y es muchísimo más rápido

In [128]:
#comprobar excepcionismo
excepcionismo #vacía BIEN!!

[]

In [129]:
tags_music = pd.DataFrame(listatag)

In [130]:
tags_music.head()

,albumartist,artist,genre,album,year,track,track_total,title,duration,filesize,audio_offset,channels,comment,composer,disc_total,samplerate,archivo,ruta,ruta_total
0,Alex Turner,Alex Turner,None,Submarine,2011,1,None,Stuck On The Puzzle (Intro),54.199780,2184408,16907.0,2.0,,None,None,44100.0,01 Stuck On The Puzzle (Intro).mp3,"../../../Music/Britpop, Indie y rock alt britá...","../../../Music/Britpop, Indie y rock alt britá..."
1,Alex Turner,Alex Turner,None,Submarine,2011,2,None,Hiding Tonight,187.200656,7502914,16895.0,2.0,,None,None,44100.0,02 Hiding Tonight.mp3,"../../../Music/Britpop, Indie y rock alt britá...","../../../Music/Britpop, Indie y rock alt britá..."
2,Alex Turner,Alex Turner,None,Submarine,2011,3,None,Glass In The Park,240.166035,9620931,16901.0,2.0,,None,None,44100.0,03 Glass In The Park.mp3,"../../../Music/Britpop, Indie y rock alt britá...","../../../Music/Britpop, Indie y rock alt britá..."
3,Alex Turner,Alex Turner,None,Submarine,2011,4,None,It's Hard To Get Around The Wind,247.508929,9914593,16931.0,2.0,,None,None,44100.0,04 It's Hard To Get Around The Wind.mp3,"../../../Music/Britpop, Indie y rock alt britá...","../../../Music/Britpop, Indie y rock alt britá..."
4,Alex Turner,Alex Turner,None,Submarine,2011,5,None,Stuck On The Puzzle,211.553851,8476774,16905.0,2.0,,None,None,44100.0,05 Stuck On The Puzzle.mp3,"../../../Music/Britpop, Indie y rock alt britá...","../../../Music/Britpop, Indie y rock alt britá..."


In [131]:
#podríamos incluir toda esta info en una basse de datos
#utlizaremos mongo ahora


## Guardar info en base de datos
- MONGO

In [136]:
client = MongoClient("localhost:27017")
client.list_database_names()

['admin',
 'carpeta_music',
 'config',
 'discogs',
 'lastusers',
 'local',
 'maps',
 'musicable']

In [146]:
# usamos carpeta_music que tiene dos colecciones (archivos y tags) para guardar la info recopilada con walk y tinytag
carpeta_music = client.get_database("carpeta_music")

In [148]:
carpeta_music.list_collection_names()

['archivos', 'tags']

In [149]:
 # iteramos sobre los archivos de la carpeta Music y guardamos su info en coleccion archivos
for (path, ficheros, archivos) in walk("../../../Music"):
    dicc = {}
    dicc['path'] =  path
    dicc['ficheros'] = ficheros
    dicc['archivos'] = archivos
    carpeta_music.archivos.insert_one(dicc)

In [150]:
#iteramos por dataframe y guradraos info en coleccion tags
for i,r in archivos_mus.iterrows():
    
    try:

        dicc = {}
        #tag = TinyTag.get({str(r.ruta)}+'/'+{str(r.archivo)})
        tag = TinyTag.get(r.ruta+"/"+r.archivo)
        dicc['albumartist'] = tag.albumartist
        dicc['artist'] = tag.artist
        dicc['genre'] = tag.genre
        dicc['album'] = tag.album
        dicc['year'] = tag.year
        dicc['track'] = tag.track    
        dicc['track_total'] = tag.track_total
        dicc['title'] = tag.title    
        dicc['duration'] = tag.duration
        dicc['filesize'] = tag.filesize
        dicc['audio_offset'] = tag.audio_offset
        dicc['channels'] = tag.channels
        dicc['comment'] = tag.comment
        dicc['composer'] = tag.composer
        dicc['disc_total'] = tag.disc_total
        #dicc['extra'] = tag.extra
        dicc['samplerate'] = tag.samplerate        
        dicc['archivo'] = r.archivo
        dicc['ruta'] = r.ruta        
        dicc['ruta_total'] = f'{r.ruta}/{r.archivo}'

        carpeta_music.tags.insert_one(dicc)

        if i%10000 == 0:
            print(i, 'done', r.ruta, r.archivo)
    except Exception as e:
        print(e)
        print(f'cant be done: {i}, {r.ruta+"/"+r.archivo}')
        excepcionismo.append(i)
print('done, bitches')

10000 done ../../../Music/Flamenco, Fusión, Música del mundo, ritmos latinos y Francia/La cabra mecánica/1997 Cuando me suenan las tripas 02 Pégate un tiro.mp3
20000 done ../../../Music/Pop, Pop Dance, Club/Rihanna/2007 Good Girl Gone Bad 02 Push Up on Me.mp3
done, bitches
